# Advanced Sacred Geometry: Layered Composition

This notebook demonstrates advanced layered composition techniques using the sacred geometry library. The composition module allows you to create complex hierarchical arrangements of sacred geometric forms with the following features:

- **Hierarchical Scene Graph**: Create parent-child relationships between shapes
- **Transformations**: Apply position, rotation, and scale to shapes
- **Sacred Ratios**: Apply proportions based on sacred ratios (φ, √2, √3, etc.)
- **Symmetry Operations**: Create forms with different symmetry groups
- **Fractal Patterns**: Generate recursive, self-similar patterns

Let's explore these capabilities!

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from functools import partial

# Import our sacred geometry modules
from sacred_geometry.shapes.shapes import (
    create_tetrahedron, create_cube, create_octahedron,
    create_icosahedron, create_dodecahedron, create_merkaba,
    create_cuboctahedron, create_torus
)
from sacred_geometry.visualization.visualization import plot_3d_shape
from sacred_geometry.composition.composition import (
    GeometryNode, GeometryScene, 
    create_symmetry_group, create_fractal_shape, create_shape_mandala,
    create_geometric_progression, apply_golden_ratio_proportions,
    rotate_around_axis, mirror_across_plane, apply_radial_symmetry,
    PHI, SQRT2, SQRT3, PI
)

# Configure matplotlib for better notebook display
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 10)

## 1. Visualizing a Geometry Node

Let's first create a helper function to visualize any GeometryNode and its children:

In [ ]:
def plot_geometry_node(node, ax=None, show_axes=True, title=None, display_info=True):
    """Plot a GeometryNode and all its children."""
    # Create figure if not provided
    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
    
    # Function to plot a single node
    def plot_node(node):
        if not node.visible:
            return
            
        # Get the transformed shape data
        shape_data = node.get_transformed_shape()
        if shape_data is None:
            return
            
        # Handle different shape types
        if 'tetrahedron1' in shape_data and 'tetrahedron2' in shape_data:
            # Handle Merkaba (Star Tetrahedron) case
            for tetra_key, alpha_mod in zip(['tetrahedron1', 'tetrahedron2'], [0, -0.1]):
                tetra = shape_data[tetra_key]
                vertices = tetra['vertices']
                faces = tetra['faces']
                
                face_collection = []
                for face in faces:
                    face_vertices = [vertices[i] for i in face]
                    face_collection.append(face_vertices)
                
                # Adjust alpha for second tetrahedron
                alpha = max(0.1, node.alpha + alpha_mod)
                
                ax.add_collection3d(Poly3DCollection(
                    face_collection, 
                    color=node.color, 
                    alpha=alpha, 
                    linewidths=1,
                    edgecolors='black'
                ))
        elif 'vertices' in shape_data and 'triangular_faces' in shape_data and 'square_faces' in shape_data:
            # Handle shapes with both triangular and square faces (like cuboctahedron)
            vertices = shape_data['vertices']
            
            # Triangular faces
            tri_faces = shape_data['triangular_faces']
            tri_face_collection = []
            for face in tri_faces:
                face_vertices = [vertices[i] for i in face]
                tri_face_collection.append(face_vertices)
            
            ax.add_collection3d(Poly3DCollection(
                tri_face_collection, 
                color=node.color, 
                alpha=node.alpha, 
                linewidths=1,
                edgecolors='black'
            ))
            
            # Square faces
            square_faces = shape_data['square_faces']
            square_face_collection = []
            for face in square_faces:
                face_vertices = [vertices[i] for i in face]
                square_face_collection.append(face_vertices)
            
            # Use slightly different alpha for square faces
            ax.add_collection3d(Poly3DCollection(
                square_face_collection, 
                color=node.color, 
                alpha=max(0.1, node.alpha - 0.1), 
                linewidths=1,
                edgecolors='black'
            ))
        elif 'vertices' in shape_data:
            # Handle regular shapes with vertices and faces
            vertices = shape_data['vertices']
            faces = shape_data['faces']
            
            face_collection = []
            for face in faces:
                face_vertices = [vertices[i] for i in face]
                face_collection.append(face_vertices)
            
            ax.add_collection3d(Poly3DCollection(
                face_collection, 
                color=node.color, 
                alpha=node.alpha, 
                linewidths=1,
                edgecolors='black'
            ))
    
    # Recursively plot node and all its children
    def plot_node_recursive(node):
        # Plot node itself
        plot_node(node)
        
        # Plot all children
        for child in node.children:
            plot_node_recursive(child)
    
    # Start plotting from root node
    plot_node_recursive(node)
    
    # Compute appropriate axis limits
    all_vertices = []
    
    def collect_vertices(node):
        shape_data = node.get_transformed_shape()
        if shape_data is None:
            return
            
        if 'tetrahedron1' in shape_data and 'tetrahedron2' in shape_data:
            all_vertices.extend(shape_data['tetrahedron1']['vertices'])
            all_vertices.extend(shape_data['tetrahedron2']['vertices'])
        elif 'vertices' in shape_data:
            all_vertices.extend(shape_data['vertices'])
            
        for child in node.children:
            collect_vertices(child)
    
    collect_vertices(node)
    
    if all_vertices:
        # Convert to numpy array for easier manipulation
        all_vertices = np.array(all_vertices)
        
        # Compute bounds
        min_bounds = np.min(all_vertices, axis=0)
        max_bounds = np.max(all_vertices, axis=0)
        center = (min_bounds + max_bounds) / 2
        max_range = np.max(max_bounds - min_bounds) / 2 * 1.2  # Add 20% margin
        
        # Set equal axis limits
        ax.set_xlim(center[0] - max_range, center[0] + max_range)
        ax.set_ylim(center[1] - max_range, center[1] + max_range)
        ax.set_zlim(center[2] - max_range, center[2] + max_range)
    else:
        # Default limits if no vertices found
        ax.set_xlim(-2, 2)
        ax.set_ylim(-2, 2)
        ax.set_zlim(-2, 2)
    
    # Set labels and title
    if show_axes:
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
    else:
        ax.set_axis_off()
        
    if title:
        ax.set_title(title)
    else:
        ax.set_title(node.name)
    
    # Display information about the composition
    if display_info:
        def count_nodes(node):
            count = 1  # Count this node
            for child in node.children:
                count += count_nodes(child)
            return count
            
        total_nodes = count_nodes(node)
        depth = 0
        
        def max_depth(node, current_depth):
            if not node.children:
                return current_depth
            return max(max_depth(child, current_depth + 1) for child in node.children)
            
        max_tree_depth = max_depth(node, 0)
        
        info_text = f"Total nodes: {total_nodes}, Max depth: {max_tree_depth}"
        plt.figtext(0.5, 0.01, info_text, ha='center')
    
    plt.tight_layout()
    return ax

## 2. Basic Composition with Parent-Child Relationships

Let's start by creating a simple composition with parent-child relationships. We'll create a tetrahedron with smaller tetrahedra at its vertices:

In [ ]:
# Create the parent tetrahedron
tetra_data = create_tetrahedron(center=(0, 0, 0), radius=1.0)
parent_tetra = GeometryNode("Parent Tetrahedron", tetra_data, color='blue', alpha=0.3)

# Create child tetrahedra at the vertices of the parent
vertices = tetra_data['vertices']
for i, vertex in enumerate(vertices):
    child_data = create_tetrahedron(center=(0, 0, 0), radius=0.25)
    child = GeometryNode(f"Child_{i}", child_data, 
                       color='red', alpha=0.7, 
                       position=tuple(vertex))
    # Add as a child of the parent tetrahedron
    parent_tetra.add_child(child)

# Visualize the composition
plot_geometry_node(parent_tetra, title="Parent-Child Composition")

## 3. Applying Sacred Ratios in Compositions

Let's create a more complex composition using sacred ratios. We'll create a geometric progression based on the golden ratio:

In [ ]:
# Create a golden ratio progression with octahedra
octa_data = create_octahedron(center=(0, 0, 0), radius=1.0)
phi_progression = create_geometric_progression(
    octa_data, 
    count=7, 
    ratio='phi',
    direction=(1, 0.5, 0.25),  # Direction vector
    base_color='purple'
)

# Visualize the progression
plot_geometry_node(phi_progression, title="Golden Ratio (φ) Progression")

## 4. Creating a Composition with Multiple Symmetry Types

Now let's create a more complex composition that combines different symmetry types:

In [ ]:
# Create a scene to hold our composition
scene = GeometryScene()

# Create a tetrahedral symmetry group with tetrahedra
tetrahedral_group = create_symmetry_group(
    create_tetrahedron,
    symmetry_type='tetrahedral',
    scale=0.4,
    color='red',
    alpha=0.6
)

# Create an octahedral symmetry group with octahedra
octahedral_group = create_symmetry_group(
    create_octahedron,
    symmetry_type='octahedral',
    scale=0.35,
    color='blue',
    alpha=0.4
)

# Create a central icosahedron
icosa_data = create_icosahedron(center=(0, 0, 0), radius=0.25)
central_icosa = GeometryNode("Central Icosahedron", icosa_data, color='gold', alpha=0.7)

# Add all groups to the scene
root_node = GeometryNode("Multi-Symmetry Composition")
root_node.add_child(tetrahedral_group)
root_node.add_child(octahedral_group)
root_node.add_child(central_icosa)

# Visualize the composition
plot_geometry_node(root_node, title="Multi-Symmetry Sacred Geometry Composition")

## 5. Creating a Sacred Geometry Mandala

Let's create a mandala-like arrangement of sacred geometry shapes:

In [ ]:
# Create a mandala with tetrahedra
tetra_mandala = create_shape_mandala(
    create_tetrahedron,
    rings=3,
    shapes_in_first_ring=8,
    spiral=False,
    scale_factor=0.7
)

# Visualize the mandala
plot_geometry_node(tetra_mandala, title="Sacred Geometry Tetrahedron Mandala")

Now let's create a 3D spiral mandala using the height increment:

In [ ]:
# Create a spiral mandala with merkabas
spiral_mandala = create_shape_mandala(
    create_merkaba,
    rings=4,
    shapes_in_first_ring=6,
    spiral=True,
    scale_factor=0.8,
    height_increment=0.3
)

# Visualize the spiral mandala
plot_geometry_node(spiral_mandala, title="3D Spiral Merkaba Mandala")

## 6. Creating Fractal Geometry

Now let's create a fractal composition where shapes are recursively added at the vertices of their parents:

In [ ]:
# Create a fractal tetrahedron
fractal_tetra = create_fractal_shape(
    create_tetrahedron,
    iterations=3,
    scale_factor=0.4,
    base_color='green'
)

# Visualize the fractal tetrahedron
plot_geometry_node(fractal_tetra, title="Fractal Tetrahedron Composition")

## 7. Advanced Composition: The Cosmic Tree

Let's create a more complex structure combining multiple sacred geometry principles into a "Cosmic Tree" structure:

In [ ]:
# Create the cosmic tree
def create_cosmic_tree():
    # Root node
    cosmic_tree = GeometryNode("Cosmic Tree")
    
    # Create the trunk (a cuboctahedron)
    trunk_data = create_cuboctahedron(center=(0, 0, 0), radius=1.0)
    trunk = GeometryNode("Trunk", trunk_data, color='brown', alpha=0.6)
    cosmic_tree.add_child(trunk)
    
    # Create branches using the golden ratio
    branch_directions = [
        (1, 0, 1),    # Upper right
        (-1, 0, 1),   # Upper left
        (0, 1, 1),    # Front upper
        (0, -1, 1),   # Back upper
        (1, 1, 0),    # Right side
        (-1, 1, 0),   # Left side
        (1, -1, 0),   # Right back
        (-1, -1, 0)   # Left back
    ]
    
    for i, direction in enumerate(branch_directions):
        # Normalize direction vector
        norm = np.sqrt(sum(d*d for d in direction))
        direction = tuple(d/norm for d in direction)
        
        # Create a branch endpoint position
        branch_length = 1.0 + (i % 3) * 0.3  # Vary branch lengths slightly
        pos = tuple(d * branch_length for d in direction)
        
        # Create a branch (an octahedron)
        branch_data = create_octahedron(center=(0, 0, 0), radius=0.3)
        branch = GeometryNode(f"Branch_{i}", branch_data, 
                            color='green', alpha=0.7, 
                            position=pos)
        
        # Rotate to point outward
        import math
        theta = math.atan2(direction[1], direction[0])
        phi = math.acos(direction[2]) if norm > 0 else 0
        branch.set_rotation((0, phi, theta))
        
        trunk.add_child(branch)
        
        # Add fruits to the branches (small icosahedra)
        fruit_count = 3
        for j in range(fruit_count):
            # Calculate fruit positions along the branch
            t = (j + 1) / (fruit_count + 1)  # Parametric position along branch
            fruit_pos = tuple(t * p for p in pos)
            
            # Create fruit
            fruit_data = create_icosahedron(center=(0, 0, 0), radius=0.15)
            fruit = GeometryNode(f"Fruit_{i}_{j}", fruit_data, 
                               color='gold', alpha=0.8, 
                               position=fruit_pos)
            
            # Scale fruit by the golden ratio based on position
            fruit.scale = 0.7 / (1 + (j * 0.4))
            trunk.add_child(fruit)
    
    # Create a root system below (mirroring the branches but inverted)
    root_directions = [
        (0.7, 0, -1),    # Down right
        (-0.7, 0, -1),   # Down left
        (0, 0.7, -1),    # Down front
        (0, -0.7, -1),   # Down back
    ]
    
    for i, direction in enumerate(root_directions):
        # Normalize and scale
        norm = np.sqrt(sum(d*d for d in direction))
        direction = tuple(d/norm for d in direction)
        
        # Root length
        root_length = 1.2
        pos = tuple(d * root_length for d in direction)
        
        # Create a root (tetrahedron)
        root_data = create_tetrahedron(center=(0, 0, 0), radius=0.25)
        root = GeometryNode(f"Root_{i}", root_data, 
                          color='brown', alpha=0.6, 
                          position=pos)
        
        # Point downward
        root.set_rotation((np.pi, 0, 0))
        trunk.add_child(root)
        
    # Add a merkaba at the very top
    merkaba_data = create_merkaba(center=(0, 0, 0), radius=0.4, rotation=np.pi/4)
    merkaba = GeometryNode("Crown", merkaba_data, 
                         color='purple', alpha=0.7, 
                         position=(0, 0, 2.0))
    trunk.add_child(merkaba)
    
    return cosmic_tree

# Create and visualize the cosmic tree
cosmic_tree = create_cosmic_tree()
plot_geometry_node(cosmic_tree, title="The Cosmic Tree of Sacred Geometry")

## 8. Interactive Composition Explorer

Let's create an interactive viewer that allows us to explore different compositions:

In [ ]:
def interactive_composition_explorer():
    # Available compositions to explore
    compositions = {
        "Phi Progression": lambda: create_geometric_progression(
            create_tetrahedron(center=(0, 0, 0), radius=1.0), 
            count=6, ratio='phi', direction=(1, 0, 0)
        ),
        "Fractal Tetrahedron": lambda: create_fractal_shape(
            create_tetrahedron, iterations=3, scale_factor=0.4
        ),
        "Tetrahedron Mandala": lambda: create_shape_mandala(
            create_tetrahedron, rings=3, shapes_in_first_ring=8
        ),
        "Spiral Merkaba Mandala": lambda: create_shape_mandala(
            create_merkaba, rings=3, shapes_in_first_ring=6, 
            spiral=True, height_increment=0.3
        ),
        "Tetrahedral Symmetry": lambda: create_symmetry_group(
            create_tetrahedron, symmetry_type='tetrahedral', scale=0.6
        ),
        "Octahedral Symmetry": lambda: create_symmetry_group(
            create_octahedron, symmetry_type='octahedral', scale=0.6
        ),
        "Icosahedral Symmetry": lambda: create_symmetry_group(
            create_icosahedron, symmetry_type='icosahedral', scale=0.4
        ),
        "Dihedral-5 Symmetry": lambda: create_symmetry_group(
            create_cube, symmetry_type='dihedral-5', scale=0.4
        ),
        "Cosmic Tree": create_cosmic_tree
    }
    
    # Function to update the plot
    def update_plot(composition_name):
        clear_output(wait=True)
        
        # Create the selected composition
        composition = compositions[composition_name]()
        
        # Plot it
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        plot_geometry_node(composition, ax=ax, title=composition_name)
        plt.show()
        
        # Display the dropdown again
        display(composition_dropdown)
    
    # Create the dropdown
    composition_dropdown = widgets.Dropdown(
        options=list(compositions.keys()),
        value=list(compositions.keys())[0],
        description='Composition:',
        style={'description_width': 'initial'}
    )
    
    # Add observer
    composition_dropdown.observe(lambda change: update_plot(change['new']), names='value')
    
    # Display initial state
    display(composition_dropdown)
    update_plot(composition_dropdown.value)

# Launch the interactive explorer
interactive_composition_explorer()

## 9. Creating a Complex Merkaba-Vector Equilibrium Composition

Let's create a more complex composition that builds on the Merkaba and Vector Equilibrium relationship we explored in the previous notebook:

In [ ]:
def create_merkaba_ve_complex():
    # Create the root node
    composition = GeometryNode("Merkaba-Vector Equilibrium Complex")
    
    # Create central Vector Equilibrium
    ve_data = create_cuboctahedron(center=(0, 0, 0), radius=1.0)
    ve = GeometryNode("Central Vector Equilibrium", ve_data, color='gold', alpha=0.3)
    composition.add_child(ve)
    
    # Create central Merkaba aligned with the VE
    merkaba_data = create_merkaba(center=(0, 0, 0), radius=1.0, rotation=np.pi/4)
    merkaba = GeometryNode("Central Merkaba", merkaba_data, color='blue', alpha=0.5)
    # Scale by 1/φ for perfect sacred proportion alignment
    merkaba.scale = 1.0 / PHI
    composition.add_child(merkaba)
    
    # Add 12 smaller Merkabas at the 12 vertices of the Vector Equilibrium
    vertices = ve_data['vertices']
    for i, vertex in enumerate(vertices):
        # Create a Merkaba at this vertex
        child_merkaba_data = create_merkaba(center=(0, 0, 0), radius=0.2, rotation=np.pi/4)
        child_merkaba = GeometryNode(f"Merkaba_{i}", child_merkaba_data, 
                                 color='purple', alpha=0.6,
                                 position=tuple(vertex))
        
        # Rotate each Merkaba to face outward from center
        # Calculate angles based on vertex position
        x, y, z = vertex
        theta = np.arctan2(y, x)  # Angle in xy-plane
        phi = np.arccos(z / np.linalg.norm(vertex)) if np.linalg.norm(vertex) > 0 else 0  # Angle from z-axis
        
        # Set the rotation - rotate around y first, then z
        child_merkaba.set_rotation((0, phi, theta))
        
        # Add to composition
        composition.add_child(child_merkaba)
    
    # Add a golden spiral of tetrahedra
    spiral_count = 8
    for i in range(spiral_count):
        # Calculate the spiral position
        angle = 0.4 * i  # Angle in radians
        radius = 1.0 + (0.2 * i)  # Increasing radius
        height = -1.0 + (0.3 * i)  # Increasing height
        
        # Convert to cartesian coordinates
        x = radius * np.cos(angle)
        y = radius * np.sin(angle)
        z = height
        
        # Create a tetrahedron at this position
        tetra_data = create_tetrahedron(center=(0, 0, 0), radius=0.25)
        tetra = GeometryNode(f"Spiral_Tetra_{i}", tetra_data,
                          color='green', alpha=0.7,
                          position=(x, y, z))
        
        # Scale each tetrahedron by a power of 1/φ
        tetra.scale = 0.8 / (PHI ** (i * 0.5))
        
        # Set rotation to follow the spiral
        tetra.set_rotation((0, 0, angle))
        
        # Add to composition
        composition.add_child(tetra)
    
    # Add an upper toroidal ring
    torus_positions = []
    torus_count = 12
    torus_radius = 1.8
    torus_height = 1.5
    
    for i in range(torus_count):
        angle = 2 * np.pi * i / torus_count
        x = torus_radius * np.cos(angle)
        y = torus_radius * np.sin(angle)
        z = torus_height
        torus_positions.append((x, y, z))
    
    for i, pos in enumerate(torus_positions):
        # Create a cuboctahedron at this position
        cuboct_data = create_cuboctahedron(center=(0, 0, 0), radius=0.3)
        cuboct = GeometryNode(f"Torus_Cuboct_{i}", cuboct_data,
                           color='cyan', alpha=0.5,
                           position=pos)
        
        # Point slightly outward
        x, y, z = pos
        theta = np.arctan2(y, x)
        cuboct.set_rotation((0, np.pi/6, theta))
        
        # Add to composition
        composition.add_child(cuboct)
    
    return composition

# Create and visualize
merkaba_ve_complex = create_merkaba_ve_complex()
plot_geometry_node(merkaba_ve_complex, title="Merkaba-Vector Equilibrium Complex")

## 10. Custom Composition Exercise

Here's a space for you to create your own custom sacred geometry composition using the tools we've built. Here's a template to start with:

In [ ]:
def create_my_custom_composition():
    # Create the root node
    composition = GeometryNode("My Custom Sacred Geometry Composition")
    
    # Add your shapes and structures here
    # Example:
    icosa_data = create_icosahedron(center=(0, 0, 0), radius=1.0)
    icosa = GeometryNode("Central Icosahedron", icosa_data, color='blue', alpha=0.4)
    composition.add_child(icosa)
    
    # Add more elements to create your unique composition
    # ...
    
    return composition

# Create and visualize your composition
my_composition = create_my_custom_composition()
plot_geometry_node(my_composition)

## Conclusion

In this notebook, we've explored the advanced layered composition capabilities of our sacred geometry library. These tools allow for creating complex, hierarchical arrangements of sacred geometric forms while incorporating important principles like sacred ratios, symmetry, and fractal patterns.

Key concepts we've covered:

- Using a scene graph architecture for parent-child relationships
- Applying transformations (position, rotation, scale) to geometry nodes
- Incorporating sacred ratios like the Golden Ratio (φ) into compositions
- Creating symmetric arrangements according to different symmetry groups
- Building fractal and recursive geometric patterns
- Combining different sacred geometry forms into cohesive compositions

These techniques can be used to create an infinite variety of sacred geometry compositions for meditation, artistic exploration, or educational purposes.